In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [27]:
def read_dataset(test_size=0.2):

    data = pd.read_csv('./dataset/indian_liver_patient.csv')

    data = data.dropna()

    data['Gender'] = [ 0 if gender == 'Male' else 1 for gender in data['Gender'] ]

    #data=(data-data.mean())/data.std() # standartization
    
    x, y = data.iloc[:, :-1].to_numpy(), data.iloc[:, -1].to_numpy()
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=test_size)

    return x_train, x_test, y_train, y_test

In [28]:
x_train, x_test, y_train, y_test = read_dataset()

In [29]:
x_train.shape, y_train.shape

((463, 10), (463,))

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_regression = LogisticRegression()
svm = SVC()

log_regression.fit(x_train, y_train)
svm.fit(x_train, y_train)


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [32]:
from sklearn.metrics import accuracy_score

y_hat_log = log_regression.predict(x_test)
y_hat_svm = svm.predict(x_test)

log_acc = accuracy_score(y_test, y_hat_log)
svm_acc = accuracy_score(y_test, y_hat_svm)

print(log_acc, svm_acc)


0.7413793103448276 0.7327586206896551


In [33]:
def get_texts(filename):
    with(open(filename, 'r', encoding='utf-8')) as texts_file:
            texts = texts_file.read().split('\n')
    return texts

In [34]:
texts = get_texts('./dataset/bmstu_vk.txt')

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

texts_tf_ifd = TfidfVectorizer().fit_transform(texts)

In [39]:
vec = texts_tf_ifd[0].toarray()

In [42]:
from sklearn.metrics import euclidean_distances

results = []

for text_vec in texts_tf_ifd:
    text_vec = text_vec.toarray()

    distance = euclidean_distances(vec, text_vec)

    results.append(distance)


In [44]:
results[:10]

[array([[1.49011612e-08]]),
 array([[1.3901302]]),
 array([[1.39351423]]),
 array([[1.39736495]]),
 array([[1.40487806]]),
 array([[1.4023892]]),
 array([[1.40737862]]),
 array([[1.39426645]]),
 array([[1.40759651]]),
 array([[1.38892426]])]

In [45]:
from gensim.models import FastText



In [46]:
texts[:5]

['вечер профсоюз сегодняшний наш герой иметь многолетний история панк рок группа калифорния быть основывать год часто относить альтернативный рок скейт панк панк рок музыка также содержать элемент гранж метать ска текст песня обычно носить оттенок сарказм посвящать личный взаимоотношение социально политический проблема в настоящий время ее состав входить вокалист гитарист декстер холланд гитарист кевин вассерман ударник пит парад год они гастролировать гитарист группа тодд морс в течение долгий время год ударный группа играть рон уэлти свой первый альбом группа выпускать год покровительство продюсер тома уилсон котор ранее работать группа альбом издавать небольшой тираж этот последовать шестинедельный тур с год лейбл записывать выпускать два работа альбом альбом становиться один самый успешный карьера записывать выпускать год сингл становиться хит приносить группа всемирный известность группа приглашать разогрев а двухлетний гастрольный тур поддержка заключать контракт лейбл который вы

In [48]:
texts_splitted = [text.split() for text in texts]

In [56]:
def pad_text(document, pad_len):
    if len(document) >= pad_len:
        return document[:pad_len]

    diff = pad_len - len(document)

    for i in range(diff):
        document.append('<PAD>')
    return document

In [57]:
pad_len = 50

padded = [pad_text(text, pad_len) for text in texts_splitted]

In [60]:
model = FastText(padded, window=3, size=10)

In [62]:
model.wv.most_similar('общага')

[('3общ', 0.9139272570610046),
 ('5общ', 0.9105619788169861),
 ('общяга', 0.9007397890090942),
 ('ага', 0.8961151242256165),
 ('комната', 0.8638366460800171),
 ('бумага', 0.8521593809127808),
 ('общежитие', 0.8421720862388611),
 ('черно', 0.8382362127304077),
 ('че', 0.8321846723556519),
 ('мп', 0.8287723064422607)]

In [63]:
texts_fast = [ np.mean(model[word])  for text in padded for word in text ] 

In [64]:
texts_fast[:5]

[-0.24456295, -0.51390254, 0.32241684, -1.4395494, -0.055223804]